# Отток клиентов

**Дано:**  из «Бета-Банка» стали уходить клиенты. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых. Есть исторические данные о поведении клиентов и расторжении договоров с банком.

**Задача:** спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Построить модель с предельно большим значением F1-меры до 0.59. Проверить F1-меру на тестовой выборке самостоятельно. Дополнительно измерить AUC-ROC, сравнивая её значение с F1-мерой


**План исследования:**
 1. Обзор данных.
 2. Предобработка данных.
 3. Исследование баланса классов, обучение модель без учёта дисбаланса.
 4. Улучшение качества модели с учетом дисбаланса классов. Лучшая модель.
 5. Финальное тестирование.
 6. Вывод

## 1. Обзор данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    clients = pd.read_csv('/Users/galina//Desktop/учёба/Churn.csv')
except:
    clients = pd.read_csv('/datasets/Churn.csv')

In [3]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
clients.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
#смотрю, есть ли дубликаты: 
clients.duplicated().sum()

0

In [6]:
#сразу посмотрю доли объектов 0 и 1 в целевом признаке, чтобы понять, увеличивать их потом или уменьшать:
print("Доля объектов положительного класса в целевом признаке = {:.2f}".format(sum(clients['Exited']/len(clients))))
print("Доля объектов отрицательного класса в целевом признаке = {:.2f}".format(1 - sum(clients['Exited']/len(clients))))

Доля объектов положительного класса в целевом признаке = 0.20
Доля объектов отрицательного класса в целевом признаке = 0.80


### Выводы по п.1. Обзор данных:

1. В таблице 10000 объектов. Пропуски есть в столбце `Tenure`. Явных дубликатов нет.
2. Можно избавиться от столбца `RowNumber`, так как он фактически дублирует нумерацию объектов с шагом 1, а также от столбцов `CustomerId` и `Surname`, так как значения в них не влияют на дальнейший анализ.
3. Можно изменить названия стобцов - перевести в строчные и заменить пробелы _.
4. Можно перевести данные в столбце `Tenure` из типа float в int.
5. Предварительно можно преобразовать данные в столбце `Gender` и `Geography`.
6. В целевом признаке высокая доля объектов отрицательного класса, то есть в дальнейшем нужно будет увеличить долю объектов положительного класса.

## 2. Предобработка данных

In [7]:
#избавляюсь от столбцов:
clients = clients.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [8]:
#заменяю заглавные буквы в названиях столбцов на строчные:
clients.columns = clients.columns.str.lower()
clients.columns

Index(['creditscore', 'geography', 'gender', 'age', 'tenure', 'balance',
       'numofproducts', 'hascrcard', 'isactivemember', 'estimatedsalary',
       'exited'],
      dtype='object')

In [9]:
#меняю названия столбцов:
clients = clients.rename(columns={'creditscore':'credit_score',
                                  'numofproducts':'num_of_products',
                                  'hascrcard':'has_cr_card',
                                  'isactivemember':'is_active_member',
                                  'estimatedsalary': 'estimated_salary'
                                 })
clients.head(5)

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [10]:
#смотрю долю пропусков в столбце `tenure`:
f"{clients['tenure'].isna().sum() / clients['tenure'].sum():.2%}"

'2.00%'

In [11]:
#поскольку всего 2% пропусков, можно избавиться от этих объектов:
clients = clients.dropna(subset=['tenure'])

In [12]:
#перевожу данные в столбце 'tenure' в int:
clients['tenure'] = clients['tenure'].astype('int')

In [32]:
#смотрю кол-во уникальных значений в столбцах, чтобы понять,
#какие категориальные признаки можно преобразовать в численные:
for column in clients:
    print(clients[column].value_counts(normalize=True))
    print()

850    0.02310
678    0.00660
655    0.00561
705    0.00528
683    0.00517
        ...   
407    0.00011
417    0.00011
365    0.00011
440    0.00011
419    0.00011
Name: credit_score, Length: 458, dtype: float64

37    0.04785
35    0.04741
38    0.04664
34    0.04532
33    0.04433
       ...   
82    0.00011
88    0.00011
85    0.00011
92    0.00011
83    0.00011
Name: age, Length: 70, dtype: float64

1     0.104719
2     0.104499
8     0.102629
3     0.102079
5     0.101969
7     0.101749
4     0.097349
9     0.097019
6     0.096909
10    0.049060
0     0.042020
Name: tenure, dtype: float64

0.00         0.361126
105473.74    0.000220
130170.82    0.000220
108935.39    0.000110
114453.58    0.000110
               ...   
176024.05    0.000110
107499.70    0.000110
98807.45     0.000110
136596.85    0.000110
75075.31     0.000110
Name: balance, Length: 5807, dtype: float64

1    0.507865
2    0.460235
3    0.025740
4    0.006160
Name: num_of_products, dtype: float64

1    0.704983
0 

In [14]:
#применяю прямое кодирование к столбцам `geography` и `gender`:
gender_ohe = pd.get_dummies(clients['gender'], drop_first=True)
geography_ohe = pd.get_dummies(clients['geography'], drop_first=True)
clients.drop(['gender', 'geography'], axis=1, inplace=True)
clients_ohe = pd.concat([clients, gender_ohe, geography_ohe], axis=1)

In [15]:
clients_ohe.head(5)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,Male,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


### Выводы по п.2. Предобработка данных:
1. Избавилась от верблюжьего стиля и пробелов в заголовках, избавилась от пропусков и ненужных столбцов.
2. Заменила типы данных на более подходящие.
3. Применила прямое кодирование к столбцам geography и gender.

## 3. Исследование баланса классов, обучение модели без учёта дисбаланса

In [16]:
#создаю переменные для признаков и целевого признака:
features = clients_ohe.drop(['exited'], axis=1)
target = clients_ohe['exited']

#разделяю данные на обучающую и валидационную выборки в соотношении 60/40:
features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              target, test_size=0.4,
                                                                              random_state=12345)

#разделяю валидационную выборку на валидационную и тестовую в соотношении 20/20:
features_test, features_valid, target_test, target_valid = train_test_split(features_valid,
                                                                            target_valid, test_size=0.5,
                                                                            random_state=12345)

#смотрю размеры получившихся выборок:
for i in [features_train, features_valid, target_train, target_valid, features_test, target_test]:
    print(i.shape)

(5454, 11)
(1819, 11)
(5454,)
(1819,)
(1818, 11)
(1818,)


### Обучение моделей без учёта дисбаланса

In [17]:
#дерево решений:
best_f1_dtc = 0
best_depth_dtc = 0
best_auc_roc = 0
for depth in range(1, 15):
    dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    dtc.fit(features_train, target_train)
    predicted_valid_dtc = dtc.predict(features_valid)
    f1_dtc = f1_score(target_valid, predicted_valid_dtc)
    auc_roc_dtc = roc_auc_score(target_valid, dtc.predict_proba(features_valid)[:, 1])
    print("F1 = {:.3f}, AUC-ROC = {:.3f}".format(f1_dtc, auc_roc_dtc), 'на шаге', depth)
    if f1_dtc > best_f1_dtc:
        best_f1_dtc = f1_dtc
        best_depth_dtc = depth
        best_auc_roc = auc_roc_dtc
        
print()
print('F1 для дерева решений =', best_f1_dtc)
print('Максимальная глубина =', best_depth_dtc)
print('AUC-ROC =', best_auc_roc)

F1 = 0.000, AUC-ROC = 0.690 на шаге 1
F1 = 0.509, AUC-ROC = 0.740 на шаге 2
F1 = 0.413, AUC-ROC = 0.803 на шаге 3
F1 = 0.526, AUC-ROC = 0.823 на шаге 4
F1 = 0.482, AUC-ROC = 0.827 на шаге 5
F1 = 0.528, AUC-ROC = 0.832 на шаге 6
F1 = 0.556, AUC-ROC = 0.831 на шаге 7
F1 = 0.499, AUC-ROC = 0.798 на шаге 8
F1 = 0.515, AUC-ROC = 0.786 на шаге 9
F1 = 0.516, AUC-ROC = 0.763 на шаге 10
F1 = 0.534, AUC-ROC = 0.755 на шаге 11
F1 = 0.491, AUC-ROC = 0.721 на шаге 12
F1 = 0.517, AUC-ROC = 0.703 на шаге 13
F1 = 0.513, AUC-ROC = 0.704 на шаге 14

F1 для дерева решений = 0.5559322033898305
Максимальная глубина = 7
AUC-ROC = 0.8305629303585707


In [18]:
#логистическая регрессия:
sr = RobustScaler()
features_train_sr = sr.fit_transform(features_train)
features_valid_sr = sr.transform(features_valid)

lgrg = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
lgrg.fit(features_train_sr, target_train)
predicted_valid_lgrg = lgrg.predict(features_valid_sr)
print('F1 для логистической регрессии =', f1_score(target_valid, predicted_valid_lgrg))
print('AUC-ROC для логистической регрессии =', roc_auc_score(target_valid, lgrg.predict_proba(features_valid)[:, 1]))

F1 для логистической регрессии = 0.32119914346895073
AUC-ROC для логистической регрессии = 0.5


In [19]:
#случайный лес:
best_f1_rfc = 0
best_depth_rfc = 0
best_est_rfc = 0
best_roc_auc_score_rfc = 0
for depth in range(1, 10):
    for est in range(1, 100):
        rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        rfc.fit(features_train, target_train)
        predicted_valid_rfc = rfc.predict(features_valid)
        f1_rfc = f1_score(target_valid, predicted_valid_rfc)
        roc_auc_score_rfc = roc_auc_score(target_valid, rfc.predict_proba(features_valid)[:,1])
        if f1_rfc > best_f1_rfc:
            best_f1_rfc = f1_rfc
            best_depth_rfc = depth
            best_est_rfc = est
            best_roc_auc_score_rfc = roc_auc_score_rfc
            
print('F1 для случайного леса =', best_f1_rfc)
print('Максимальная глубина =', best_depth_rfc)
print('Число деревьев =', best_est_rfc)
print('AUC-ROC =', best_roc_auc_score_rfc)

F1 для случайного леса = 0.5595667870036101
Максимальная глубина = 9
Число деревьев = 13
AUC-ROC = 0.8476433622891388


### Выводы по п.3. Исследование баланса классов, обучение модели без учёта дисбаланса:

Без учёта дисбаланса наибольшее значение F1 наблюдается в моделях дерево решений и случайный лес - 0.555 и 0.559 соответственно. Проверю далее обе, так как разница между F1 небольшая.

## 4. Улучшение качества модели с учетом дисбаланса классов. Лучшая модель.

### 4.1. Увеличение выборки 

In [20]:
#увеличиваю число наблюдений положительного класса:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled


features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

На этом этапе мне подсказали, что есть такой модуль GridSearchCV, который подбирает параметры для моделей. Остальное подглядела тут - https://vc.ru/ml/147132-kak-avtomaticheski-podobrat-parametry-dlya-modeli-mashinnogo-obucheniya-ispolzuem-gridsearchcv и хотела применить к увеличенной выборке, но вывод был очень долгий. Пока применяю к увеличенной выборке модели дерево решений и случайный лес.

In [21]:
#дерево решений:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled) 
predicted_valid = model.predict(features_valid)

print('Для дерева решений на увеличенной выборке F1 =', f1_score(target_valid, predicted_valid))
print('AUC-ROC =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))


Для дерева решений на увеличенной выборке F1 = 0.4730290456431535
AUC-ROC = 0.6751292531265284


In [22]:
#случайный лес:
model = RandomForestClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled) 
predicted_valid = model.predict(features_valid)

print('Для случайного леса на увеличенной выборке F1 =', f1_score(target_valid, predicted_valid))
print('AUC-ROC =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))

Для случайного леса на увеличенной выборке F1 = 0.5972850678733032
AUC-ROC = 0.8455822989201736


### 4.1. Уменьшение выборки 

In [23]:
#попробую также сделать объекты частого класса не такими частыми:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [24]:
#дерево решений:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_downsampled, target_downsampled) 
predicted_valid = model.predict(features_valid)

print('Для дерева решений на увеличенной выборке F1 =', f1_score(target_valid, predicted_valid))
print('AUC-ROC =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))

Для дерева решений на увеличенной выборке F1 = 0.5118934348239772
AUC-ROC = 0.7363925180682674


In [25]:
#случайный лес:
model = RandomForestClassifier(random_state=12345)
model.fit(features_downsampled, target_downsampled) 
predicted_valid = model.predict(features_valid)

print('Для случайного леса на увеличенной выборке F1 =', f1_score(target_valid, predicted_valid))
print('AUC-ROC =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))

Для случайного леса на увеличенной выборке F1 = 0.5556711758584808
AUC-ROC = 0.852862005791161


Увеличение выборки справилось лучше.

### Выводы по п.4. Улучшение качества модели с учетом дисбаланса классов. Лучшая модель:
Лучшая модель по показателям F1 и AUC-ROC - случайный лес.

## 5. Финальное тестирование.

In [29]:
#объединяю для обучения данные train и valid:
features_train_valid = pd.concat([features_train] + [features_valid])
target_train_valid = pd.concat([target_train] + [target_valid]) 

In [30]:
#применяю upsampling к объединенной выборке:
features_upsampled, target_upsampled = upsample(features_train_valid, target_train_valid, 4)

In [31]:
#тестирую случайный лес:
model_rfc_test = RandomForestClassifier(random_state=12345)
model_rfc_test.fit(features_upsampled, target_upsampled)
predictions_rfc_test = model_rfc_test.predict(features_test)

print('F1 =', f1_score(target_test, predictions_rfc_test))
print('AUC-ROC =', roc_auc_score(target_test, model_rfc_test.predict_proba(features_test)[:,1]))

F1 = 0.6020558002936858
AUC-ROC = 0.8675378320021647


## 6. Вывод

Случайный лес с дисбалансом показывал F1 = 56%, AUC-ROC = 83%.
<br>После увеличения числа наблюдений положительного класса показатель F1 вырос и достиг 59,7%, AUC-ROC = 84%.
<br>Проверка на тестовой выборке подтвердила, что upsampling помог справиться с дисбалансом классов и удалось увеличить F1 до 60%, AUC-ROC до 86%.
<br>Поскольку F1 - среднее гармоническое полноты и точности (растет только вместе с полнотой и точностью), то увеличение F1 свидетельствует об изменении в алгоритме в лучшую сторону. То есть вероятность ухода клиента из банка меньше вероятности того, что он останется (клиент скорее жив, чем ушел).